# Pull funds from a particular legislator and make a plot

In [3]:
#load everything
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from tqdm import tqdm

from operator import attrgetter

dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)

Populating the interactive namespace from numpy and matplotlib


# Step 1: Get Bill and sponsors from bill

In [4]:
#assume given bill number, type, and congress
bill_number = '4387'
bill_type = 'hr'
congress = '113'

#load bill
query = "SELECT * FROM features_legis WHERE bill_type LIKE '"+bill_type+"' AND bill_number LIKE '"+bill_number+"' AND congress = '"+congress+"' ;" #should only be one match.
bill_legis = pd.read_sql_query(query,engine) #note to self, bills repeat, I only want the most recent one.
#drop useless columns.
bill_legis = bill_legis.drop(['bill_type','bill_number','congress','num_amends','index','status','result','final_result'],1)
#read in legislator info
legis = pd.read_sql_table('sponsor_list_det',engine)

In [5]:
cols = bill_legis.columns.tolist()
mat = bill_legis.as_matrix().flatten().tolist()

In [6]:
print(len(mat))
print(len(cols))

869
869


In [7]:
#list of legislators on bill
legislators = list()
for leg in range(0,len(cols)):
    if mat[leg] == 1:
        legislators.append(cols[leg])

In [8]:
print(legislators)

['Rep,"Bachus, Spencer",AL', 'Rep,"Cotton, Tom",AR', 'Rep,"Fincher, Stephen Lee",TN', 'Rep,"Garrett, Scott",NJ', 'Rep,"Hultgren, Randy",IL', 'Rep,"Hurt, Robert",VA', 'Rep,"Mulvaney, Mick",SC', 'Rep,"Peters, Gary C.",MI', 'Rep,"Pittenger, Robert",NC', 'Rep,"Ross, Dennis A.",FL', 'Rep,"Rothfus, Keith J.",PA', 'Rep,"Stivers, Steve",OH', 'Rep,"Wagner, Ann",MO', 'Rep,"Westmoreland, Lynn A.",GA']


In [7]:
#loop through legislators and get funding df for each one, stored in a list.
legis_funds = list()
legis_names = list()
for leg in legislators:
    #first get row in legis
    legis_ind = legis[legis.sponsor == leg]
    if legis_ind['index'].size > 1: #something has gone wrong.
        print('Something has gone wrong.')
        break
    legis_funds.append(contributions.pull_contributions(legis_ind,legis,engine))
    legis_names.append(legis_ind.qsponsor.iloc[0])

Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen
Rep
Sen


In [8]:
print(len(legis_funds))
print(legis_names)
print(legis_names[0])

14
['Rep, Bachus,  Spencer', 'Rep, Cotton,  Tom', 'Rep, Fincher,  Stephen Lee', 'Rep, Garrett,  Scott', 'Rep, Hultgren,  Randy', 'Rep, Hurt,  Robert', 'Rep, Mulvaney,  Mick', 'Rep, Peters,  Gary C.', 'Rep, Pittenger,  Robert', 'Rep, Ross,  Dennis A.', 'Rep, Rothfus,  Keith J.', 'Rep, Stivers,  Steve', 'Rep, Wagner,  Ann', 'Rep, Westmoreland,  Lynn A.']
Rep, Bachus,  Spencer


In [9]:
allfunds = list()
allcontribs = list()
#do what I need with one dataframe first
counter = 0
for df in legis_funds :
    #sum up contributions from the same contributers
    u_reg = np.unique(df.registrantid) #all unique contributors
    allcontribs.append(u_reg)
    u_reg_name = list() #name of contributor
    u_reg_con = list() #sum of contributions
    for rid_ind in range(0,len(u_reg)):
        regid = u_reg[rid_ind] #id for contributor
        regname = df[df.registrantid == regid]['registrantname'].iloc[0]
        tot_money = np.sum(df[df.registrantid == regid]['amount'])
        u_reg_name.append(regname)
        u_reg_con.append(tot_money)
    u_reg = u_reg.tolist()
    temp_name = legis_names[counter]
    temp_name = [temp_name]*len(u_reg)
    allfunds.append(list(zip(u_reg,u_reg_name,u_reg_con,temp_name)))
    print(allfunds[0][0])
    counter = counter+1

(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')
(385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer')


In [10]:
print(allcontribs[0])

[      385       682       693       933      1114      2274      2571
      2638      2640      2662      2731      3322      4026      4163
      4301      4766      5153      5233      5290      5357      5368
      6597      6757      7304      7441      8453      8577      9000
      9033      9170      9237      9897     10627     10764     11060
     11093     11333     11424     11776     11878     13277     13756
     14165     15097     16178     17215     17374     18158     18422
     18694     19194     19228     19230     19480     20128     20334
     20881     20994     21687     22826     22872     23714     23997
     24123     24863     25842     26284     26400     26546     26650
     26854     26865     26901     26945     27070     27274     27811
     28218     28424     28435     28468     28470     29221     29414
     29607     29802     31895     32225     32486     34158     34637
     35002     35230     36105     36914     38483     38756     39143
     3

In [11]:
#identify all shared contributors
sharedcontribs = list()
for l1 in range(0,len(allcontribs)-1) : #loop over main list
    for item in allcontribs[l1] : #loop over all id numbers in main list
        for l2 in range(l1+1,len(allcontribs)): #loop over other lists to check for id number (item)
            otherlist = allcontribs[l2]
            if item in otherlist:
                sharedcontribs.append(item)
                break

In [12]:
u_sharedcontribs = np.unique(sharedcontribs)
print(u_sharedcontribs)

[       66       157       317       330       385       454       580
       614       682       693       933      1114      1171      1182
      1238      1295      1331      1342      1364      1375      1386
      1422      1581      1648      1650      1661      1672      1774
      1785      1967      2046      2115      2172      2183      2194
      2228      2252      2274      2321      2354      2376      2412
      2445      2491      2571      2638      2640      2662      2684
      2731      2866      2877      2968      2981      3060      3093
      3242      3388      3481      3526      3572      3628      3630
      3674      3708      3867      3925      3960      4026      4163
      4208      4301      4675      4766      4790      5120      5131
      5153      5233      5290      5357      5368      5404      5483
      5506      5610      5632      5723      5734      5836      6245
      6325      6358      6484      6597      6837      6848      7053
      

In [13]:
shared_con = list() #list that will hold all contribution tuples that come from contributors that donate to other sponsors
legislator_num = 0 #used to track which legislator the funds relate to.
for rid in u_sharedcontribs : #go through registrant ids in shared contribs
    for l in allfunds: #list of all lists containing tuples of contributors
        for tup in l: #loop through each tupple in a list. #could be vectorized I bet
            if tup[0] == rid:
                shared_con.append(tup)
                break

In [14]:
print(shared_con)

[(66, 'ABBOTT LABORATORIES', 22500.0, 'Rep, Hultgren,  Randy'), (66, 'ABBOTT LABORATORIES', 26000.0, 'Rep, Hurt,  Robert'), (66, 'ABBOTT LABORATORIES', 19500.0, 'Rep, Stivers,  Steve'), (157, 'ADAMS AND REESE, LLP', 2000.0, 'Rep, Fincher,  Stephen Lee'), (157, 'Adams and Reese, LLP', 6000.0, 'Rep, Ross,  Dennis A.'), (317, 'AEROJET ROCKETDYNE, INC', 3000.0, 'Rep, Cotton,  Tom'), (317, 'AEROJET-GENERAL CORPORATION', 15500.0, 'Rep, Ross,  Dennis A.'), (330, 'EADS North America', 5000.0, 'Rep, Garrett,  Scott'), (330, 'EADS North America', 10000.0, 'Rep, Ross,  Dennis A.'), (385, 'AFLAC INCORPORATED', 3500.0, 'Rep, Bachus,  Spencer'), (385, 'AFLAC INCORPORATED', 3000.0, 'Rep, Cotton,  Tom'), (385, 'AFLAC INCORPORATED', 6000.0, 'Rep, Fincher,  Stephen Lee'), (385, 'AFLAC INCORPORATED', 11000.0, 'Rep, Garrett,  Scott'), (385, 'AFLAC INCORPORATED', 9500.0, 'Rep, Hultgren,  Randy'), (385, 'AFLAC INCORPORATED', 15000.0, 'Rep, Mulvaney,  Mick'), (385, 'AFLAC INCORPORATED', 7000.0, 'Rep, Peters,

In [15]:
#loop over shared money to get total contributed from the shared contributors.
tot_u_shared_contribs = list()
for item in range(0,len(u_sharedcontribs)):
    tot = 0
    rid = u_sharedcontribs[item]
    for tup in shared_con:
        if tup[0] == rid:
            tot = tot + tup[2]
            name = tup[1] #could be placed in boolean to only occur once, but this is still not slow.
    tot_u_shared_contribs.append((rid,name,tot))
#sort to get the largest donators of the shared contributors
tot_u_shared_contribs = sorted(tot_u_shared_contribs, key=lambda tup: tup[2], reverse=True)
print(tot_u_shared_contribs)

[(316890, 'DELOITTE LLP', 644500.0), (4766, 'ASSOCIATED BUILDERS AND CONTRACTORS, INC.', 468200.0), (19228, 'INDEPENDENT INSURANCE AGENTS & BROKERS OF AMERICA', 383354.99), (27274, 'National Beer Wholesalers Association', 310950.0), (57453, 'HONEYWELL INTERNATIONAL', 296868.57), (10764, 'PRICEWATERHOUSECOOPERS', 289500.0), (27070, 'NATIONAL ASSOCIATION OF REALTORS', 287020.0), (76855, 'DLA Piper LLP (US)', 280612.0), (26400, 'BANK OF AMERICA CORPORATION', 276500.0), (2571, 'American Hospital Association', 253100.0), (1785, 'American Bankers Association', 253000.0), (26901, 'National Association of Insurance and Financial Advisors', 239250.0), (9033, 'CME Group, Inc.', 232500.0), (39223, 'UPS (UNITED PARCEL SERVICE)', 216876.0), (400390816, 'Koch Companies Public Sector, LLC', 201500.0), (53824, 'CAPITAL ONE FINANCIAL CORPORATION', 194200.0), (11093, 'THE COUNCIL OF INSURANCE AGENTS AND BROKERS', 191500.0), (313976, 'THE GOLDMAN SACHS GROUP, INC.', 186500.0), (298034, 'ASSOCIATION FOR A

In [16]:
#plot the money distribution to individual senators by the largest contributor
rid = tot_u_shared_contribs[0][0]
name = tot_u_shared_contribs[0][1]
total = tot_u_shared_contribs[0][2]
toPlot = [x for x in shared_con if x[0] == rid]

In [17]:
print(toPlot)

[(316890, 'DELOITTE LLP', 151500.0, 'Rep, Bachus,  Spencer'), (316890, 'DELOITTE LLP', 21000.0, 'Rep, Cotton,  Tom'), (316890, 'DELOITTE LLP', 32000.0, 'Rep, Fincher,  Stephen Lee'), (316890, 'DELOITTE LLP', 52500.0, 'Rep, Garrett,  Scott'), (316890, 'DELOITTE LLP', 14500.0, 'Rep, Hultgren,  Randy'), (316890, 'DELOITTE LLP', 55000.0, 'Rep, Hurt,  Robert'), (316890, 'DELOITTE LLP', 9000.0, 'Rep, Mulvaney,  Mick'), (316890, 'DELOITTE LLP', 112000.0, 'Rep, Peters,  Gary C.'), (316890, 'DELOITTE LLP', 4500.0, 'Rep, Pittenger,  Robert'), (316890, 'DELOITTE LLP', 62000.0, 'Rep, Ross,  Dennis A.'), (316890, 'DELOITTE LLP', 6500.0, 'Rep, Rothfus,  Keith J.'), (316890, 'DELOITTE LLP', 62500.0, 'Rep, Stivers,  Steve'), (316890, 'DELOITTE LLP', 19000.0, 'Rep, Wagner,  Ann'), (316890, 'DELOITTE LLP', 42500.0, 'Rep, Westmoreland,  Lynn A.')]


In [24]:
contrib_df = pd.DataFrame(shared_con, columns=["rid","reg_name","amount","legis"])

In [25]:
contrib_df

,rid,reg_name,amount,legis
0,66,ABBOTT LABORATORIES,22500.0,"Rep, Hultgren, Randy"
1,66,ABBOTT LABORATORIES,26000.0,"Rep, Hurt, Robert"
2,66,ABBOTT LABORATORIES,19500.0,"Rep, Stivers, Steve"
3,157,"ADAMS AND REESE, LLP",2000.0,"Rep, Fincher, Stephen Lee"
4,157,"Adams and Reese, LLP",6000.0,"Rep, Ross, Dennis A."
5,317,"AEROJET ROCKETDYNE, INC",3000.0,"Rep, Cotton, Tom"
6,317,AEROJET-GENERAL CORPORATION,15500.0,"Rep, Ross, Dennis A."
7,330,EADS North America,5000.0,"Rep, Garrett, Scott"
8,330,EADS North America,10000.0,"Rep, Ross, Dennis A."
9,385,AFLAC INCORPORATED,3500.0,"Rep, Bachus, Spencer"


In [30]:
legis = np.unique(contrib_df["legis"])
rids = np.unique(contrib_df["rid"])

In [29]:
legis

array(['Rep, Bachus,  Spencer', 'Rep, Cotton,  Tom',
       'Rep, Fincher,  Stephen Lee', 'Rep, Garrett,  Scott',
       'Rep, Hultgren,  Randy', 'Rep, Hurt,  Robert',
       'Rep, Mulvaney,  Mick', 'Rep, Peters,  Gary C.',
       'Rep, Pittenger,  Robert', 'Rep, Ross,  Dennis A.',
       'Rep, Rothfus,  Keith J.', 'Rep, Stivers,  Steve',
       'Rep, Wagner,  Ann', 'Rep, Westmoreland,  Lynn A.'], dtype=object)

In [43]:
shared_con_inf = list()
influence_df = pd.DataFrame()
for leg in tqdm(legis):
    for rid in rids:
        reg_name = contrib_df[contrib_df.rid == rid]["reg_name"].iloc[0]
        cons = contrib_df[(contrib_df.legis == leg) & (contrib_df.rid == rid)]
        other_cons = contrib_df[(contrib_df.legis == leg)]
        if(len(cons)) == 0:
            inf = 0.
        else:
            amount = cons['amount'].iloc[0] #should only be one row in cons
            other_amount = sum(other_cons['amount'].tolist())
            inf = amount/other_amount #percent influence of this organization as compared to other shared contributors.
        inf_tup = (rid,reg_name,inf,leg)
        shared_con_inf.append(inf_tup)    

100%|██████████| 14/14 [00:19<00:00,  1.41s/it]


In [44]:
#loop over shared money to get total contributed from the shared contributors.
tot_u_shared_contribs_inf = list()
for item in range(0,len(u_sharedcontribs)):
    tot = 0
    rid = u_sharedcontribs[item]
    for tup in shared_con_inf:
        if tup[0] == rid:
            tot = tot + tup[2]
            name = tup[1] #could be placed in boolean to only occur once, but this is still not slow.
    tot_u_shared_contribs_inf.append((rid,name,tot))
#sort to get the largest donators of the shared contributors
tot_u_shared_contribs_inf = sorted(tot_u_shared_contribs_inf, key=lambda tup: tup[2], reverse=True)
print(tot_u_shared_contribs_inf)

[(4766, 'ASSOCIATED BUILDERS AND CONTRACTORS, INC.', 0.4099769042968886), (316890, 'DELOITTE LLP', 0.37155310044280909), (19228, 'INDEPENDENT INSURANCE AGENTS & BROKERS OF AMERICA', 0.29004586239075003), (27274, 'National Beer Wholesalers Association', 0.2180890405073013), (26400, 'BANK OF AMERICA CORPORATION', 0.21424950639123125), (1785, 'American Bankers Association', 0.18616621837051392), (27070, 'NATIONAL ASSOCIATION OF REALTORS', 0.18110169687886266), (57453, 'HONEYWELL INTERNATIONAL', 0.1750283264181316), (400390816, 'Koch Companies Public Sector, LLC', 0.17171409272604909), (10764, 'PRICEWATERHOUSECOOPERS', 0.16676974235164246), (9033, 'CME Group, Inc.', 0.16486167848539501), (26901, 'NATIONAL ASSOCIATION OF INSURANCE AND FINANCIAL ADVISORS', 0.15992528114470278), (298034, 'ASSOCIATION FOR ADVANCED LIFE UNDERWRITING', 0.14529429061012064), (39223, 'UPS (UNITED PARCEL SERVICE)', 0.13492861295867969), (53824, 'CAPITAL ONE FINANCIAL CORPORATION', 0.13438916270233492), (313976, 'TH

In [32]:
bleh = contrib_df[(contrib_df.rid == 0)]

In [34]:
print(len(bleh))

0


In [100]:
#output to intermediate file
x = list()
y = list()
for i in toPlot:
    x.append(i[3])
    y.append(i[2])
with open("./d3_test/data.tsv", "w") as record_file:
    record_file.write("Legislator\tContribution\n")
    for i in range(0,len(x)):
        record_file.write(str(x[i])+"\t"+str(y[i])+"\n")